## Inception V3

In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input, Flatten, Dense
from sklearn.cross_validation import train_test_split
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
K.set_image_dim_ordering('th')
K.set_floatx('float32')

import pandas as pd
import numpy as np
np.random.seed(17)

train_data = np.load('train.npy')
train_target = np.load('train_target.npy')

x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.25, random_state=32)

Using TensorFlow backend.


In [2]:
input = Input(shape=(3, 150, 150))
base_model = InceptionV3(weights=None, input_tensor=input, include_top=False)
base_output = base_model.output
x = Flatten(name='flat')(base_output)
predictions = Dense(3, activation='softmax',name='output_layer')(x)
model = Model(input=base_model.input, output=predictions)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ou..., inputs=Tensor("in...)`
  


In [3]:
# print the model structure
for i, layer in enumerate(model.layers):
    print(i, layer.name)

(0, 'input_1')
(1, 'conv2d_1')
(2, 'batch_normalization_1')
(3, 'activation_1')
(4, 'conv2d_2')
(5, 'batch_normalization_2')
(6, 'activation_2')
(7, 'conv2d_3')
(8, 'batch_normalization_3')
(9, 'activation_3')
(10, 'max_pooling2d_1')
(11, 'conv2d_4')
(12, 'batch_normalization_4')
(13, 'activation_4')
(14, 'conv2d_5')
(15, 'batch_normalization_5')
(16, 'activation_5')
(17, 'max_pooling2d_2')
(18, 'conv2d_9')
(19, 'batch_normalization_9')
(20, 'activation_9')
(21, 'conv2d_7')
(22, 'conv2d_10')
(23, 'batch_normalization_7')
(24, 'batch_normalization_10')
(25, 'activation_7')
(26, 'activation_10')
(27, 'average_pooling2d_1')
(28, 'conv2d_6')
(29, 'conv2d_8')
(30, 'conv2d_11')
(31, 'conv2d_12')
(32, 'batch_normalization_6')
(33, 'batch_normalization_8')
(34, 'batch_normalization_11')
(35, 'batch_normalization_12')
(36, 'activation_6')
(37, 'activation_8')
(38, 'activation_11')
(39, 'activation_12')
(40, 'mixed0')
(41, 'conv2d_16')
(42, 'batch_normalization_16')
(43, 'activation_16')
(44, 'c

In [4]:
# frozen the first 310 layers
for layer in model.layers[:310]:
    layer.trainable = False
for layer in model.layers[310:]:
    layer.trainable = True


model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
datagen = ImageDataGenerator(rotation_range=0.9, zoom_range=0.5, shear_range=0.3, \
                             horizontal_flip=True, vertical_flip=True)
datagen.fit(train_data)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=15, shuffle=True), steps_per_epoch=len(x_train)/100, \
                    epochs=100, verbose=1, validation_data=(x_val_train, y_val_train))

Epoch 1/100


InternalError: Failed to create session.